# 250225-2

## 1. Load

In [3]:
from v250224_2 import * 
import pickle

In [4]:
# .pkl 파일에서 불러오기
with open("./data/data.pkl", "rb") as f:
    data_loaded = pickle.load(f)

# 변수 개별 할당
y = data_loaded["y"]
yU = data_loaded["yU"]
yP = data_loaded["yP"]
t = data_loaded["t"]
regions = data_loaded["regions"]

## 2. Fit 

In [5]:
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, filters):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvGRU(node_features, filters, 2)
        self.linear = torch.nn.Linear(filters, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h
model = RecurrentGCN(node_features=24, filters=16)  # node_features = LAGS
model_u = RecurrentGCN(node_features=4, filters=16)  # node_features = LAGS
model_p = RecurrentGCN(node_features=24, filters=16)  # node_features = LAGS

In [5]:
yhat = split_fit_merge_stgcn(
    FX = y,
    train_ratio = 0.8,     
    model = model, 
    lags = 24, 
    epoch = 5, 
    dataset_name = None
)

1/5
2/5
3/5
4/5
5/5


In [6]:
yUhat, yPhat = split_fit_merge_eptstgcn(
    FXs = (yU, yP),
    train_ratio = 0.8, 
    models = (model_u, model_p),
    lags = (4,24),
    epochs = (5,5),
    dataset_name = None
)

1/5
1/5
2/5
2/5
3/5
3/5
4/5
4/5
5/5
5/5


## 3. Save

In [8]:
# 데이터 분할
train_ratio = 0.8
total_time_steps = len(t)
train_size = int(np.floor(total_time_steps * train_ratio))
test_size = total_time_steps - train_size
t_train, t_test = t[:train_size], t[train_size:] if test_size > 0 else None
y_train, y_test = y[:train_size, :], y[train_size:, :] if test_size > 0 else None
yhat_train, yhat_test = yhat[:train_size, :], yhat[train_size:, :] if test_size > 0 else None
yUhat_train, yUhat_test = yUhat[:train_size, :], yUhat[train_size:, :] if test_size > 0 else None
yPhat_train, yPhat_test = yPhat[:train_size, :], yPhat[train_size:, :] if test_size > 0 else None

# 훈련 데이터 및 테스트 데이터 스택 쌓기
train_data_stacked = np.stack((yhat_train, yUhat_train, yPhat_train), axis=0)
test_data_stacked = np.stack((yhat_test, yUhat_test, yPhat_test), axis=0)

# 저장할 파일 이름 설정 (모형 이름과 시뮬레이션 번호 반영)
filename_train = f'./results/GConvGRU_train.npy'
filename_test = f'./results/GConvGRU_test.npy'

# NumPy 파일로 저장
np.save(filename_train, train_data_stacked)
np.save(filename_test, test_data_stacked)